In [1]:
import pandas as pd
import json

In [6]:
reg = pd.read_csv(f"./CleanedData/Income_Inequality_Regression_Ready.csv").iloc[3:]
ine = pd.read_csv(f"./CleanedData/Income_Inequality_Changes.csv").iloc[3:]


In [7]:
reg

,Destination,income_High,income_Low,income_Mid,0-16,17-34,35-54,55-66,67+,High_0-16,...,Mid_17-34,High_35-54,Low_35-54,Mid_35-54,High_55-66,Low_55-66,Mid_55-66,High_67+,Low_67+,Mid_67+
3,Ashfield_2016,1889.2002,4637.4928,5302.2870,1088.6612,2602.7868,1530.9564,477.9276,6128.6480,159.6117,...,1153.1595,241.0499,595.2654,694.6411,70.6221,192.2306,215.0749,971.7306,2411.3144,2745.6030
4,Ashford_2016,7516.5753,3168.9387,2898.9331,1247.8744,2783.2789,1795.3832,658.8096,7099.1010,721.1703,...,630.2439,988.3480,418.7651,388.2701,389.3573,151.8552,117.5971,3917.8701,1679.0623,1502.1686
5,Babergh_2016,2053.3002,1458.5515,5228.5152,832.3289,1660.8627,1170.9878,476.1372,4600.0503,183.7596,...,934.0859,244.6785,183.5648,742.7445,106.6758,70.8074,298.6540,1075.4108,770.3628,2754.2767
6,Barking and Dagenham_2016,21211.4723,1669.1483,4219.0244,3489.3517,5924.1243,3469.4847,520.9366,13695.7477,2807.0743,...,1022.3412,2774.6026,178.0990,516.7831,415.0563,22.8656,83.0147,10726.1234,847.3245,2122.2998
7,Barnet_2016,37321.1251,3320.4264,8735.8414,3612.4328,13470.8554,5977.3020,1134.6243,25182.1784,2969.5601,...,2814.3607,4928.7135,267.3233,781.2652,854.8563,84.1004,195.6676,19062.9884,1687.1882,4432.0018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,Wrexham / Wrecsam_2024,3086.5249,3236.9416,2031.9905,509.7987,2147.8474,1020.2783,340.6935,4336.8391,165.2033,...,504.3378,337.3071,426.8945,256.0767,101.2934,140.9133,98.4868,1596.9426,1690.2055,1049.6910
2876,Wychavon_2024,4536.8840,4719.6659,4852.5885,1050.9698,2827.2396,1743.5979,1011.1696,7476.1615,276.6867,...,975.1182,549.5931,592.0376,601.9672,309.7937,406.5818,294.7941,2374.2639,2546.4968,2555.4008
2877,Wyre_2024,3429.2857,6433.0227,4096.2627,1134.9555,2440.4056,1617.2333,1273.5345,7492.4422,211.1510,...,747.6944,372.1284,789.8398,455.2651,279.7882,609.0667,384.6796,1820.2942,3471.8748,2200.2732
2878,Wyre Forest_2024,2824.5723,4192.8225,1715.1503,669.5458,1914.3929,1032.5310,540.9155,4575.1599,197.8721,...,462.0661,310.9024,532.5722,189.0564,140.1060,338.8016,62.0079,1469.2563,2221.7179,884.1857


In [8]:
ine

,Unnamed: 0,council,year,fw_index,fw_change
3,3,Birmingham,2019,4.119175,-0.097304
4,4,Birmingham,2020,4.155534,0.036358
5,5,Birmingham,2021,3.884944,-0.270590
6,6,Birmingham,2022,3.799673,-0.085271
7,7,Birmingham,2023,3.928850,0.129177
...,...,...,...,...,...
415,415,Yorkshire and The Humber,2021,3.831163,0.035373
416,416,Yorkshire and The Humber,2022,3.659636,-0.171527
417,417,Yorkshire and The Humber,2023,3.576946,-0.082691
418,418,Yorkshire and The Humber,2024,3.427897,-0.149049
